# Notebook 06 — Monitoreo de Drift y Reentrenamiento (MLOps)

Este notebook implementa una capa de monitoreo para el sistema predictivo:

- Data drift (features)
- Score drift
- Label drift
- Performance decay
- Drift por segmentos
- Drift ponderado por importancia
- Semáforo ejecutivo
- Trigger de reentrenamiento

Outputs en: outputs/monitoring/

In [24]:
from pathlib import Path
import json
from datetime import datetime

import numpy as np
import pandas as pd
import joblib

SEED = 42
np.random.seed(SEED)

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 160)

In [25]:
RAIZ = Path.cwd()

RUTA_DATASET = RAIZ / "data" / "processed" / "azure_pm" / "dataset_modelo.parquet"
RUTA_MODELO  = RAIZ / "modelos" / "modelo_baseline_falla_30d.joblib"

DIR_MONITORING = RAIZ / "outputs" / "monitoring"
DIR_MONITORING.mkdir(parents=True, exist_ok=True)

dataset = pd.read_parquet(RUTA_DATASET).sort_values(["fecha","machineID"])
artefacto = joblib.load(RUTA_MODELO)

modelo = artefacto["modelo_calibrado"]
columnas_features = artefacto["columnas_features"]
objetivo = artefacto["objetivo"]

print("Dataset:", dataset.shape)


Dataset: (31567, 71)


In [26]:
DIAS_REFERENCIA = 60
DIAS_PRODUCCION = 14

fecha_max = dataset["fecha"].max()

inicio_prd = fecha_max - pd.Timedelta(days=DIAS_PRODUCCION-1)
fin_prd = fecha_max

fin_ref = inicio_prd - pd.Timedelta(days=1)
inicio_ref = fin_ref - pd.Timedelta(days=DIAS_REFERENCIA-1)

ref = dataset[(dataset["fecha"]>=inicio_ref)&(dataset["fecha"]<=fin_ref)].copy()
prd = dataset[(dataset["fecha"]>=inicio_prd)&(dataset["fecha"]<=fin_prd)].copy()

print("Referencia:", ref.shape)
print("Producción:", prd.shape)


Referencia: (3670, 71)
Producción: (222, 71)


In [27]:
ref["score"] = modelo.predict_proba(ref[columnas_features])[:,1]
prd["score"] = modelo.predict_proba(prd[columnas_features])[:,1]

In [28]:
def calcular_psi(a, b, n_bins=10):

    a = pd.Series(a).dropna()
    b = pd.Series(b).dropna()

    if len(a)<50 or len(b)<50:
        return np.nan

    cortes = np.unique(np.quantile(a, np.linspace(0,1,n_bins+1)))

    if len(cortes)<3:
        return np.nan

    a_c,_ = np.histogram(a, bins=cortes)
    b_c,_ = np.histogram(b, bins=cortes)

    a_p = a_c/max(a_c.sum(),1)
    b_p = b_c/max(b_c.sum(),1)

    eps = 1e-6
    a_p = np.clip(a_p,eps,None)
    b_p = np.clip(b_p,eps,None)

    return float(np.sum((b_p-a_p)*np.log(b_p/a_p)))


def calcular_ks(a,b):

    a = pd.Series(a).dropna().values
    b = pd.Series(b).dropna().values

    if len(a)<50 or len(b)<50:
        return np.nan

    a = np.sort(a)
    b = np.sort(b)

    v = np.sort(np.unique(np.concatenate([a,b])))

    cdf_a = np.searchsorted(a,v,side="right")/len(a)
    cdf_b = np.searchsorted(b,v,side="right")/len(b)

    return float(np.max(np.abs(cdf_a-cdf_b)))


In [29]:
columnas_num = [c for c in columnas_features if pd.api.types.is_numeric_dtype(dataset[c])]

filas=[]

for col in columnas_num:
    filas.append({
        "feature":col,
        "psi":round(calcular_psi(ref[col],prd[col]),4),
        "ks":round(calcular_ks(ref[col],prd[col]),4),
        "media_ref":round(ref[col].mean(),2),
        "media_prd":round(prd[col].mean(),2)
    })

reporte_drift_features = pd.DataFrame(filas)\
    .sort_values("psi",ascending=False)

reporte_drift_features.head(10)


,feature,psi,ks,media_ref,media_prd
49,volt_mean_roll14d_mean,0.3601,0.2655,-0.08,0.31
51,rotate_mean_roll14d_mean,0.2836,0.1816,0.02,-0.15
41,volt_mean_roll7d_mean,0.2513,0.2262,-0.06,0.29
16,age,0.2325,0.1139,0.03,0.15
54,pressure_mean_roll14d_std,0.2272,0.1932,0.02,0.15
33,volt_mean_roll3d_mean,0.1815,0.1568,-0.02,0.44
53,pressure_mean_roll14d_mean,0.1667,0.1725,0.04,0.13
50,volt_mean_roll14d_std,0.1505,0.1506,-0.06,0.09
43,rotate_mean_roll7d_mean,0.1318,0.1843,-0.00,-0.11
52,rotate_mean_roll14d_std,0.1293,0.0908,0.01,0.04


In [30]:
psi_score = round(calcular_psi(ref["score"],prd["score"]),4)
ks_score = round(calcular_ks(ref["score"],prd["score"]),4)

reporte_drift_scores = pd.DataFrame([{
    "psi_score":psi_score,
    "ks_score":ks_score,
    "mean_ref":round(ref["score"].mean(),4),
    "mean_prd":round(prd["score"].mean(),4)
}])

reporte_drift_scores


,psi_score,ks_score,mean_ref,mean_prd
0,0.1014,0.095,0.5739,0.5839


In [32]:
from sklearn.metrics import roc_auc_score, average_precision_score


def perf_periodos(df,freq="W"):

    df=df.copy()
    df["periodo"]=df["fecha"].dt.to_period(freq).astype(str)

    out=[]

    for p,g in df.groupby("periodo"):

        y=g[objetivo]
        s=g["score"]

        auc=np.nan
        if y.nunique()>1:
            auc=roc_auc_score(y,s)

        ap=average_precision_score(y,s)

        out.append({
            "periodo":p,
            "filas":len(g),
            "rate_pct":round(y.mean()*100,2),
            "auc":round(auc,4) if auc==auc else np.nan,
            "ap":round(ap,4)
        })

    return pd.DataFrame(out)


reporte_performance = perf_periodos(prd)
reporte_performance


,periodo,filas,rate_pct,auc,ap
0,2015-12-14/2015-12-20,70,100.0,NaN,1.0
1,2015-12-21/2015-12-27,123,100.0,NaN,1.0
2,2015-12-28/2016-01-03,29,100.0,NaN,1.0


In [33]:
UMBRAL_PSI_SCORE = 0.25
UMBRAL_PSI_FEATURE = 0.20
UMBRAL_N_FEATURES = 8
UMBRAL_CAIDA_AP = 0.05
UMBRAL_CAMBIO_RATE = 10


ap_ref = average_precision_score(ref[objetivo],ref["score"])
ap_prd = reporte_performance["ap"].dropna().iloc[-1]

rate_ref = ref[objetivo].mean()*100
rate_prd = prd[objetivo].mean()*100

n_feat_drift = (reporte_drift_features["psi"]>=UMBRAL_PSI_FEATURE).sum()

alertas = {
    "psi_score":psi_score,
    "ap_ref":round(ap_ref,4),
    "ap_prd":round(ap_prd,4),
    "rate_ref":round(rate_ref,2),
    "rate_prd":round(rate_prd,2),
    "n_feat_drift":int(n_feat_drift),
    "alerta_score":psi_score>=UMBRAL_PSI_SCORE,
    "alerta_feat":n_feat_drift>=UMBRAL_N_FEATURES,
    "alerta_ap":ap_prd<=ap_ref-UMBRAL_CAIDA_AP,
    "alerta_rate":abs(rate_prd-rate_ref)>=UMBRAL_CAMBIO_RATE
}

alertas


{'psi_score': 0.1014,
 'ap_ref': 0.8438,
 'ap_prd': np.float64(1.0),
 'rate_ref': np.float64(77.25),
 'rate_prd': np.float64(100.0),
 'n_feat_drift': 5,
 'alerta_score': False,
 'alerta_feat': np.False_,
 'alerta_ap': np.False_,
 'alerta_rate': np.True_}

In [34]:
# Segmentación por modelo o age si existe

segmentos=[c for c in ["model","age","age_bin"] if c in dataset.columns]

if segmentos:

    seg=segmentos[0]

    filas=[]

    for s in ref[seg].dropna().unique():

        r=ref[ref[seg]==s]
        p=prd[prd[seg]==s]

        if len(r)<300 or len(p)<300:
            continue

        psi=calcular_psi(r["score"],p["score"])

        filas.append({
            "segmento":s,
            "psi_score":round(psi,4),
            "n_ref":len(r),
            "n_prd":len(p)
        })

    reporte_psi_segmentos=pd.DataFrame(filas)

else:
    reporte_psi_segmentos=pd.DataFrame()

reporte_psi_segmentos.head()


""


In [36]:
from sklearn.inspection import permutation_importance
from sklearn.metrics import average_precision_score

# Muestra para que no sea lento
ref_s = ref.sample(min(8000, len(ref)), random_state=SEED).copy()

# Scorer compatible con cualquier versión: (estimator, X, y) -> score
def scorer_ap(estimator, X, y):
    proba = estimator.predict_proba(X)[:, 1]
    return average_precision_score(y, proba)

perm = permutation_importance(
    estimator=modelo,
    X=ref_s[columnas_features],
    y=ref_s[objetivo],
    scoring=scorer_ap,
    n_repeats=5,
    random_state=SEED
)

importancias = (
    pd.DataFrame({
        "feature": columnas_features,
        "importance": perm.importances_mean
    })
    .sort_values("importance", ascending=False)
    .reset_index(drop=True)
)

# Normalización 0-1 (evitando división por cero)
max_imp = importancias["importance"].max()
importancias["norm"] = (importancias["importance"] / max_imp).clip(0, 1) if max_imp != 0 else 0.0

drift_ponderado = (
    reporte_drift_features
    .merge(importancias[["feature", "importance", "norm"]], on="feature", how="left")
)

drift_ponderado["drift_pond"] = (drift_ponderado["psi"] * drift_ponderado["norm"]).round(4)

drift_ponderado.sort_values("drift_pond", ascending=False).head(10)


,feature,psi,ks,media_ref,media_prd,importance,norm,drift_pond
3,age,0.2325,0.1139,0.03,0.15,0.015502,0.285152,0.0663
6,pressure_mean_roll14d_mean,0.1667,0.1725,0.04,0.13,0.015367,0.282658,0.0471
1,rotate_mean_roll14d_mean,0.2836,0.1816,0.02,-0.15,0.004362,0.080230,0.0228
29,rotate_mean_roll3d_mean,0.0657,0.1281,-0.03,-0.24,0.017599,0.323711,0.0213
5,volt_mean_roll3d_mean,0.1815,0.1568,-0.02,0.44,0.004578,0.084205,0.0153
49,vibration_mean,0.0330,0.0490,0.01,0.21,0.023681,0.435588,0.0144
18,rotate_mean_tendencia_7d,0.0937,0.0812,-0.04,-0.19,0.006650,0.122321,0.0115
42,pressure_mean,0.0373,0.0391,0.05,0.07,0.016645,0.306172,0.0114
12,volt_mean,0.1100,0.1266,-0.01,0.46,0.005439,0.100038,0.0110
16,rotate_mean,0.0983,0.1030,-0.04,-0.25,0.005739,0.105556,0.0104


In [39]:
# === Semáforo ejecutivo ===

total_alertas = sum([
    alertas["alerta_score"],
    alertas["alerta_feat"],
    alertas["alerta_ap"],
    alertas["alerta_rate"]
])

if total_alertas >= 3:
    estado = "ROJO"
    accion = "Investigar drift + validar performance; considerar reentrenamiento inmediato."
elif total_alertas == 2:
    estado = "AMARILLO"
    accion = "Monitoreo reforzado; revisar features críticas; preparar retraining si persiste."
else:
    estado = "VERDE"
    accion = "Operación estable; continuar monitoreo normal."


semaforo = {
    "estado": estado,
    "accion_recomendada": accion,
    "total_alertas": int(total_alertas),
    "detalle_alertas": {
        "score_drift": bool(alertas["alerta_score"]),
        "features_drift": bool(alertas["alerta_feat"]),
        "performance_decay": bool(alertas["alerta_ap"]),
        "label_drift": bool(alertas["alerta_rate"]),
    }
}

semaforo


{'estado': 'VERDE',
 'accion_recomendada': 'Operación estable; continuar monitoreo normal.',
 'total_alertas': 1,
 'detalle_alertas': {'score_drift': False,
  'features_drift': False,
  'performance_decay': False,
  'label_drift': True}}

In [41]:
# === Guardado final de reportes (JSON-safe) ===

def convertir_a_json_safe(obj):
    """
    Convierte recursivamente tipos numpy/pandas a tipos serializables por json.
    """
    import numpy as np
    import pandas as pd

    # numpy scalars (np.bool_, np.float64, etc.)
    if isinstance(obj, np.generic):
        return obj.item()

    # timestamps / timedeltas
    if isinstance(obj, (pd.Timestamp, pd.Timedelta)):
        return str(obj)

    # dict
    if isinstance(obj, dict):
        return {str(k): convertir_a_json_safe(v) for k, v in obj.items()}

    # list/tuple
    if isinstance(obj, (list, tuple)):
        return [convertir_a_json_safe(v) for v in obj]

    # fallback
    return obj


# CSV principales
reporte_drift_features.to_csv(DIR_MONITORING / "drift_features.csv", index=False)
reporte_drift_scores.to_csv(DIR_MONITORING / "drift_scores.csv", index=False)
reporte_performance.to_csv(DIR_MONITORING / "performance.csv", index=False)
drift_ponderado.to_csv(DIR_MONITORING / "drift_ponderado.csv", index=False)

# Segmentos (si existe)
if "reporte_psi_segmentos" in globals() and isinstance(reporte_psi_segmentos, pd.DataFrame) and len(reporte_psi_segmentos) > 0:
    reporte_psi_segmentos.to_csv(DIR_MONITORING / "drift_segmentos.csv", index=False)

# JSON ejecutivo consolidado
resumen_monitoreo = {
    "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),

    "ventana_referencia": {
        "inicio": str(inicio_ref)[:10],
        "fin": str(fin_ref)[:10],
        "filas": int(ref.shape[0]),
    },

    "ventana_produccion": {
        "inicio": str(inicio_prd)[:10],
        "fin": str(fin_prd)[:10],
        "filas": int(prd.shape[0]),
    },

    "alertas_base": alertas,
    "semaforo": semaforo,

    "metricas_score": {
        "psi_score": psi_score,
        "ks_score": ks_score,
        "mean_ref": round(float(ref["score"].mean()), 4),
        "mean_prd": round(float(prd["score"].mean()), 4),
    },

    "top_features_drift": drift_ponderado[
        ["feature", "psi", "norm", "drift_pond"]
    ].head(10).to_dict(orient="records"),
}

# Convertir a JSON-safe
resumen_monitoreo_safe = convertir_a_json_safe(resumen_monitoreo)

ruta_json = DIR_MONITORING / "resumen_monitoreo.json"
with open(ruta_json, "w", encoding="utf-8") as f:
    json.dump(resumen_monitoreo_safe, f, indent=2, ensure_ascii=False)

print("✅ Reportes guardados en:")
print(DIR_MONITORING.resolve())
print("Resumen ejecutivo:", ruta_json.resolve())

✅ Reportes guardados en:
C:\Users\sebas\OneDrive\Desktop\Proyecto Chatbot\Mantenimiento Industrial\outputs\monitoring
Resumen ejecutivo: C:\Users\sebas\OneDrive\Desktop\Proyecto Chatbot\Mantenimiento Industrial\outputs\monitoring\resumen_monitoreo.json


# Conclusiones — Notebook 06

En este notebook se implementó una capa integral de monitoreo MLOps para el sistema predictivo,
incluyendo detección de drift, seguimiento de performance y reglas automáticas de decisión.

## Estado actual del sistema

De acuerdo con las métricas calculadas y los umbrales definidos, el sistema se encuentra en estado:

**🟢 VERDE — Operación estable**

- Alertas activas: 1 de 4 (label drift).
- Score drift (PSI): 0.10 → bajo.
- Drift global de features: sin evidencia crítica.
- Performance: sin degradación significativa.

Acción recomendada:
> Continuar monitoreo normal sin necesidad de reentrenamiento inmediato.

## Análisis de drift

### Drift en scores

- PSI score: **0.10**

Este valor se encuentra bajo el umbral de alerta (0.25), indicando que la distribución
de probabilidades estimadas por el modelo se mantiene estable en el periodo reciente.

### Drift ponderado por importancia

Las variables con mayor impacto combinado (drift × importancia) fueron:

| Feature                     | PSI   | Importancia | Drift ponderado |
|-----------------------------|-------|-------------|------------------|
| age                         | 0.23  | 0.29        | 0.07             |
| rotate_mean_roll14d_mean    | 0.28  | 0.08        | 0.02             |
| volt_mean_roll7d_mean       | 0.25  | 0.02        | 0.01             |
| volt_mean_roll14d_mean      | 0.36  | 0.01        | 0.00             |
| pressure_mean_roll14d_std   | 0.23  | 0.00        | 0.00             |

En particular, la variable **age** presenta el mayor drift ponderado, combinando
cambio en distribución con alta relevancia para el modelo, por lo que debe ser
priorizada en revisiones futuras.

### Label drift

Se detectó un cambio relevante en la tasa de eventos recientes, activando
la alerta de label drift.

Este comportamiento sugiere una variación en el contexto operacional,
aunque sin impacto inmediato en la capacidad predictiva del modelo.

## Implicancias operativas

- El modelo mantiene estabilidad estadística y predictiva.
- No se observa degradación sistemática de performance.
- Existen señales tempranas en variables estructurales (edad de equipos)
  que deben ser monitoreadas.

En este contexto, no se justifica un reentrenamiento inmediato, pero sí
una vigilancia reforzada sobre las variables con mayor drift ponderado.

## Recomendaciones

1. Mantener monitoreo periódico con la misma metodología.
2. Priorizar el seguimiento de variables relacionadas con antigüedad y desgaste.
3. Revisar la evolución del label drift en las próximas ventanas.
4. Preparar proceso de reentrenamiento si se activan dos o más alertas simultáneas.

## Cierre del proyecto

Con este módulo se completa el ciclo end-to-end del sistema:

- Ingesta y preparación de datos.
- Modelado predictivo.
- Optimización operacional.
- Automatización.
- Monitoreo y gobierno del modelo.

El proyecto queda preparado para operación continua, auditoría técnica
y toma de decisiones basada en evidencia cuantitativa.